# **데이터프레임 결합**


## **1.환경준비**

* 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

* 데이터 가져오기

In [2]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")

sales1 = sales.loc[sales['Store_ID']==1]
sales2 = sales.loc[sales['Store_ID']==2]
sales5 = sales.loc[sales['Store_ID']==5]
sales11 = sales.loc[sales['Store_ID']==11]

products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_store_master.csv")

In [3]:
sales1.head()

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7
3,8,2013-01-01,1,0.0,8
4,10,2013-01-01,1,0.0,10


In [4]:
products.head()

,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price
0,20,HG001,Gardening Tools,Household Goods,2,50
1,27,HH001,Home Appliances,Household Goods,2,150
2,16,HK001,Kitchen,Household Goods,2,23
3,15,HK002,Kitchen,Household Goods,2,41
4,32,GS001,Seafood,Grocery,3,34


In [5]:
stores.head()

,Store_ID,City,State,Store_Type
0,1,Saint Paul,Minnesota,4
1,2,Saint Paul,Minnesota,4
2,3,Saint Paul,Minnesota,4
3,4,Saint Paul,Minnesota,4
4,5,Oklahoma City,Oklahoma,4


## **2.결합1 : 붙이기 concat()**

* 데이터를 생성해 봅시다.

In [62]:
df1 = pd.DataFrame({'A':[10,25], 'B':[15,30]})
df2 = pd.DataFrame({'A':[20,30, 50], 'C':[35,30, 40]})
df2.drop([1], inplace = True)

display(df1)
display(df2)

,A,B
0,10,15
1,25,30


,A,C
0,20,35
2,50,40


### **(1) 세로로 합치기 : axis = 0**

* join = 'inner'

In [8]:
pd.concat([df1, df2], axis = 0, join = 'inner')

,A
0,10
1,25
0,20
2,50


* join = 'outer'

In [9]:
pd.concat([df1, df2], axis = 0, join = 'outer')

,A,B,C
0,10,15.0,NaN
1,25,30.0,NaN
0,20,NaN,35.0
2,50,NaN,40.0


### **(2) 가로로 합치기 : axis = 1**

* join = 'inner'

In [10]:
pd.concat([df1, df2], axis = 1, join = 'inner')

,A,B,A,C
0,10,15,20,35


* join = 'outer'

In [63]:
pd.concat([df1, df2], axis = 1, join = 'outer')

,A,B,A,C
0,10.0,15.0,20.0,35.0
1,25.0,30.0,NaN,NaN
2,NaN,NaN,50.0,40.0


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] sales1,2,5,11 데이터프레임을 하나로 합칩시다. 어떻게 합치는게 좋을까요?

In [11]:
sales = pd.concat([sales1, sales2, sales5, sales11], axis=0)
saless = pd.concat([sales1, sales2, sales5, sales11], axis=0, join='outer')
# 인덱스가 다 똑같에서 inner / outer차이 없음

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.000,3
1,5,2013-01-01,1,0.000,5
2,7,2013-01-01,1,0.000,7
3,8,2013-01-01,1,0.000,8
4,10,2013-01-01,1,0.000,10
...,...,...,...,...,...
24340,53548,2013-01-31,11,10.000,22
24341,53550,2013-01-31,11,641.507,24
24342,53553,2013-01-31,11,0.000,27
24343,53555,2013-01-31,11,27.000,29


[문2] (조금 어려운 문제!)  
일별, 매장별 판매량 합계를 구해 봅시다.  
groupby, pd.concat 을 이용하여 다음과 같은 결과를 얻도록 코드를 작성해봅시다.

* 힌트
    * 1) sales1, 2, 5,11 로 부터 각각 일별 판매량 합계 집계하기
        * 집계시 .agg() 를 이용하여 열이름 변경하거나, 집계후 열이름 변경하기
    * 2) pd.concat 으로 가로로 합치기



In [28]:
s1 = sales1.groupby('Date',as_index=False)['Qty'].sum()
s2 = sales2.groupby('Date',as_index=False)['Qty'].sum()
s3 = sales5.groupby('Date',as_index=False)['Qty'].sum()
s4 = sales11.groupby('Date',as_index=False)['Qty'].sum()

#groupby와 이름변경 동시에 !!! 
#s1 = sales1.groupby('Date', as_index = False)['Qty'].agg({'qty-s1' : 'sum'})


In [29]:
s1.columns=['Date','qty_s1']
s2.columns=['Date','qty_s2']
s3.columns=['Date','qty_s3']
s4.columns=['Date','qty_s4']

In [30]:
s99 = pd.concat([s1,s2,s3,s4], axis=1, join='inner')
s99

,Date,qty_s1,Date,qty_s2,Date,qty_s3,Date,qty_s4
0,2013-01-01,0.000000,2013-01-01,0.000000,2013-01-01,0.000000,2013-01-01,0.000000
1,2013-01-02,6760.782000,2013-01-02,9221.338001,2013-01-02,9404.229020,2013-01-02,16593.312000
2,2013-01-03,5348.384001,2013-01-03,6830.089000,2013-01-03,6932.685000,2013-01-03,11032.676040
3,2013-01-04,5390.445001,2013-01-04,6742.984000,2013-01-04,6567.938000,2013-01-04,10962.687002
4,2013-01-05,5787.839000,2013-01-05,8583.055000,2013-01-05,7276.800000,2013-01-05,13315.896000
5,2013-01-06,2007.194000,2013-01-06,9317.139000,2013-01-06,9578.077030,2013-01-06,14165.615050
6,2013-01-07,5639.396000,2013-01-07,6047.629000,2013-01-07,5729.203000,2013-01-07,9946.420000
7,2013-01-08,5131.128000,2013-01-08,5545.928020,2013-01-08,5716.248000,2013-01-08,9258.956000
8,2013-01-09,6198.154980,2013-01-09,6061.471020,2013-01-09,5352.507000,2013-01-09,8644.922000
9,2013-01-10,4309.750000,2013-01-10,5070.431000,2013-01-10,5319.121020,2013-01-10,8470.169000


## **3.결합 ① : 조인 join(merge)**

* 판다스에서 join은 굉장히 간단합니다.
* 자동으로 key를 잡아줍니다
* default로 inner join을 합니다.

* ---
* merge는 한번에 2개만 되고 concat은 한번에 여러 행 가능!!

In [31]:
df1 = pd.DataFrame({'A':[1,2], 'B':[15,30], 'C':[20, 25]})
df2 = pd.DataFrame({'A':[2,3], 'D':[20, 35]})

display(df1)
display(df2)

,A,B,C
0,1,15,20
1,2,30,25


,A,D
0,2,20
1,3,35


### **(1) inner merge**

In [ ]:
#default가 inner지만 명시 
# on은 키가 되는 컬럼
pd.merge(df1, df2, how = 'inner', on = 'A')

In [32]:
pd.merge(df1, df2, how = 'inner')

,A,B,C,D
0,2,30,25,20


### **(2) left merge**

In [33]:
pd.merge(df1, df2, how = 'left')

,A,B,C,D
0,1,15,20,NaN
1,2,30,25,20.0


### **(3) right merge**

In [34]:
pd.merge(df1, df2, how = 'right')

,A,B,C,D
0,2,30.0,25.0,20
1,3,NaN,NaN,35


<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

[문1] 일별, 매장별 판매량 합계를 구해 봅시다.  
groupby, pd.merge 을 이용하여 다음과 같은 결과를 얻도록 코드를 작성해봅시다.

* 힌트
    * 1) sales1, 2, 5,11 로 부터 각각 일별 판매량 합계 집계하기
        * 집계시 .agg() 를 이용하여 열이름 변경하거나, 집계후 열이름 변경하기
    * 2) pd.merge 으로 결합하기



In [47]:
stores.head()

,Store_ID,City,State,Store_Type
0,1,Saint Paul,Minnesota,4
1,2,Saint Paul,Minnesota,4
2,3,Saint Paul,Minnesota,4
3,4,Saint Paul,Minnesota,4
4,5,Oklahoma City,Oklahoma,4


In [40]:
products.head()

,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price
0,20,HG001,Gardening Tools,Household Goods,2,50
1,27,HH001,Home Appliances,Household Goods,2,150
2,16,HK001,Kitchen,Household Goods,2,23
3,15,HK002,Kitchen,Household Goods,2,41
4,32,GS001,Seafood,Grocery,3,34


In [46]:
sales1.head()

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7
3,8,2013-01-01,1,0.0,8
4,10,2013-01-01,1,0.0,10


[문2] 매장1에 대해서, 일별, 카테고리별 판매량 합계를 구해 봅시다. 



In [53]:
temp = sales.loc[sales['Store_ID']==1]
display(temp.head())
m1 = pd.merge(temp, products, how='inner')
result = m1.groupby(['Date','Category'], as_index=False)['Qty'].sum()
result

,Order_ID,Date,Store_ID,Qty,Product_ID
0,3,2013-01-01,1,0.0,3
1,5,2013-01-01,1,0.0,5
2,7,2013-01-01,1,0.0,7
3,8,2013-01-01,1,0.0,8
4,10,2013-01-01,1,0.0,10


,Date,Category,Qty
0,2013-01-01,Drink,0.000
1,2013-01-01,Food,0.000
2,2013-01-01,Grocery,0.000
3,2013-01-01,Household Goods,0.000
4,2013-01-02,Drink,1158.000
...,...,...,...
119,2013-01-30,Household Goods,932.000
120,2013-01-31,Drink,971.000
121,2013-01-31,Food,751.766
122,2013-01-31,Grocery,1907.797


In [ ]:
# 카테고리: product, sales

[문3] 도시별, 총판매량 합계를 구하시오.  



In [59]:
tem2 = pd.merge(stores, sales)
display(m2)
tem2 = tem2.groupby(['City'],as_index=False)['Qty'].sum()

,Store_ID,City,State,Store_Type,Order_ID,Date,Qty,Product_ID
0,1,Saint Paul,Minnesota,4,3,2013-01-01,0.000,3
1,1,Saint Paul,Minnesota,4,5,2013-01-01,0.000,5
2,1,Saint Paul,Minnesota,4,7,2013-01-01,0.000,7
3,1,Saint Paul,Minnesota,4,8,2013-01-01,0.000,8
4,1,Saint Paul,Minnesota,4,10,2013-01-01,0.000,10
...,...,...,...,...,...,...,...,...
25105,54,LA,California,3,55099,2013-01-31,37.000,22
25106,54,LA,California,3,55101,2013-01-31,37.041,24
25107,54,LA,California,3,55104,2013-01-31,0.000,27
25108,54,LA,California,3,55106,2013-01-31,47.000,29


In [58]:
tem2

,City,Qty
0,Albany,1.119409e+05
1,Annapolis,2.255278e+05
2,Asheville,6.644511e+04
3,Atlanta,3.845901e+05
4,Bloomington,1.315413e+05
5,Boston,1.138053e+05
6,Charlotte,1.454050e+05
7,Columbus,1.872153e+05
8,Harrisburg,1.421269e+05
9,Indianapolis,1.488044e+05


### **(4) pivot**
* pivot : 결합은 아니지만, 집계 후 데이터프레임 구조를 변형해서 조회하는데 종종 사용 됩니다.

* 단계
    * 1) groupby
    * 2) pivot

In [60]:
# 1) 매장1의 일별 카테고리별 판매량을 집계

temp = pd.merge(sales1, products)
temp2 = temp.groupby(['Date', 'Category'], as_index = False)['Qty'].sum()
temp2

,Date,Category,Qty
0,2013-01-01,Drink,0.000
1,2013-01-01,Food,0.000
2,2013-01-01,Grocery,0.000
3,2013-01-01,Household Goods,0.000
4,2013-01-02,Drink,1158.000
...,...,...,...
119,2013-01-30,Household Goods,932.000
120,2013-01-31,Drink,971.000
121,2013-01-31,Food,751.766
122,2013-01-31,Grocery,1907.797


In [61]:
# 2) pivot
temp3 = temp2.pivot( 'Category', 'Date' ,'Qty')
temp3

C:\Users\user\AppData\Local\Temp\ipykernel_9096\4232607422.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  temp3 = temp2.pivot( 'Category', 'Date' ,'Qty')


Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,...,2013-01-22,2013-01-23,2013-01-24,2013-01-25,2013-01-26,2013-01-27,2013-01-28,2013-01-29,2013-01-30,2013-01-31
Category,,,,,,,,,,,,,,,,,,,,,
Drink,0.0,1158.000,985.000000,1055.000000,1319.000,407.000,1267.000,1115.000,1290.00000,914.000,...,1114.000,1152.000,924.000,1213.000,1132.000,417.000,830.000,999.000,1140.000,971.000
Food,0.0,1227.652,913.699000,790.366000,901.057,416.912,852.676,829.851,967.58200,775.515,...,780.201,1025.047,791.388,836.856,880.019,416.783,821.064,668.154,900.092,751.766
Grocery,0.0,3305.130,2613.685001,2711.079001,2746.782,926.282,2689.720,2356.277,3023.57298,1933.235,...,2242.216,2824.296,2221.805,2393.208,2257.907,1162.207,2208.364,2001.047,2662.485,1907.797
Household Goods,0.0,1070.000,836.000000,834.000000,821.000,257.000,830.000,830.000,917.00000,687.000,...,786.000,769.000,622.000,701.000,551.000,247.000,625.000,617.000,932.000,711.000


In [ ]:
# sns.heatmap으로 시각화 할 수도 있습니다!

plt.figure(figsize = (20, 6))
sns.heatmap(temp3)
plt.show()

## **4.복습문제**

In [65]:
# 라이브러리 불러오기
import pandas as pd

In [78]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")
stores = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_store_master.csv")

1) sales에 금액(Amt) 변수를 추가하시오.
* Amt = Qty * Price

In [76]:
sales.info()
products.info()
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25110 entries, 0 to 25109
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Order_ID    25110 non-null  int64  
 1   Date        25110 non-null  object 
 2   Store_ID    25110 non-null  int64  
 3   Qty         25110 non-null  float64
 4   Product_ID  25110 non-null  int64  
 5   Amt         15 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product_ID    15 non-null     int64 
 1   Product_Code  15 non-null     object
 2   SubCategory   15 non-null     object
 3   Category      15 non-null     object
 4   LeadTime      15 non-null     int64 
 5   Price         15 non-null     int64 
dtypes: int64(3), object(3)
memory usage: 852.0+ bytes
<class 'p

In [72]:
sp = pd.merge(sales, products)
sp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25110 entries, 0 to 25109
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order_ID      25110 non-null  int64  
 1   Date          25110 non-null  object 
 2   Store_ID      25110 non-null  int64  
 3   Qty           25110 non-null  float64
 4   Product_ID    25110 non-null  int64  
 5   Product_Code  25110 non-null  object 
 6   SubCategory   25110 non-null  object 
 7   Category      25110 non-null  object 
 8   LeadTime      25110 non-null  int64  
 9   Price         25110 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 2.1+ MB


In [73]:
sales['Amt']= sales['Qty'] * products['Price']


In [74]:
sales.head()

,Order_ID,Date,Store_ID,Qty,Product_ID,Amt
0,3,2013-01-01,1,0.0,3,0.0
1,5,2013-01-01,1,0.0,5,0.0
2,7,2013-01-01,1,0.0,7,0.0
3,8,2013-01-01,1,0.0,8,0.0
4,10,2013-01-01,1,0.0,10,0.0


2) City별 카테고리리별 매출액

In [87]:
#3개의 테이블 모두 merge해야되는 상황,, city, category, sales 
temp = pd.merge(sp, stores)
temp.head()

,Order_ID,Date,Store_ID,Qty,Product_ID,Product_Code,SubCategory,Category,LeadTime,Price,City,State,Store_Type
0,3,2013-01-01,1,0.0,3,DB001,Beverage,Drink,2,8,Saint Paul,Minnesota,4
1,1785,2013-01-02,1,1091.0,3,DB001,Beverage,Drink,2,8,Saint Paul,Minnesota,4
2,3567,2013-01-03,1,919.0,3,DB001,Beverage,Drink,2,8,Saint Paul,Minnesota,4
3,5349,2013-01-04,1,953.0,3,DB001,Beverage,Drink,2,8,Saint Paul,Minnesota,4
4,7131,2013-01-05,1,1160.0,3,DB001,Beverage,Drink,2,8,Saint Paul,Minnesota,4


In [88]:
temp.groupby(['City','Category'], as_index=False)['Qty'].sum()

,City,Category,Qty
0,Albany,Drink,28347.000000
1,Albany,Food,8863.159001
2,Albany,Grocery,55560.773004
3,Albany,Household Goods,19170.000000
4,Annapolis,Drink,43231.000000
...,...,...,...
79,Saint Paul,Household Goods,714453.000000
80,Wilmington,Drink,0.000000
81,Wilmington,Food,0.000000
82,Wilmington,Grocery,0.000000


4) 매출액이 가장 높은 주(state) top 3

In [ ]:
# 매출액 : amt
# state : sotres

,State,Qty
6,Minnesota,5.274287e+06
8,North Carolina,1.307500e+06
0,Arizona,4.475075e+05


추가 문제

* 2013년 1월 세째 주 각 도시별 subCategory별 금액 비교
    1. 2013년 1월 세째 주
        * sales['Date']를 날짜 타입으로 변환
        * week 추가
        * sales에서 week가 3인 데이터 뽑기
    2. 각 도시별 subCategory별
        * 1의 결과에서...
        * stores와 inner merge
        * products와 inner merge
    3. 도시별 subCategory별 금액
        * Qty * Price로 Amt 추가
        * groupby
    4. 좀더 보기 좋게! pivot + heatmap
        * pivot
        * heatmap

In [102]:
data = pd.read_csv('bit.ly/3WvWiOO.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'bit.ly/3WvWiOO.csv'